In [11]:
import sys
sys.version

'3.9.16 [MSC v.1931 64 bit (AMD64)]'

In [17]:
import arcpy
import os, tempfile
from urllib import request

In [18]:
# 定义工作空间
arcpy.management.CreateFileGDB(os.getcwd(), 'Live.gdb')
arcpy.env.workspace = os.path.join(os.getcwd(), 'Live.gdb')

## 下载文件

In [19]:
url = 'https://coralreefwatch.noaa.gov/product/vs/vs_polygons.json' # 珊瑚礁观测网站
temp_dir = tempfile.mkdtemp() # 定义临时目录

## 从文件创建图层

## 创建geojson 方法一

In [20]:
filename = os.path.join(temp_dir, 'latest_data.json')
response = request.urlretrieve(url, filename)
print(filename)

<Response [200]>


In [ ]:
import json
with open(filename) as json_file:
    data_raw = json.load(json_file)
    # Do something with the 'data_raw' variable
# with语句会自动关闭json文件

In [14]:
# 创建两个字典，一个用于点要素（观测站），另一个用于面要素（NOAA 确定的感兴趣区域）
# 从json文件创建geojson
data_stations = dict(type=data_raw['type'], features=[])
data_areas = dict(type=data_raw['type'], features=[])

for feat in data_raw['features']:
    if feat['geometry']['type'] == 'Point':
        data_stations['features'].append(feat)
    else: # elif feat['geometry']['type'] in ['MultiPolygon', 'Polygon']:
        data_areas['features'].append(feat)
len(data_stations['features'])


{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'adcode': 430100,
    'name': '长沙市',
    'center': [112.982279, 28.19409],
    'centroid': [113.158697, 28.224232],
    'childrenNum': 9,
    'level': 'city',
    'parent': {'adcode': 430000},
    'subFeatureIndex': 0,
    'acroutes': [100000, 430000]},
   'geometry': {'type': 'MultiPolygon',
    'coordinates': [[[[111.902613, 28.021982],
       [111.896377, 28.00871],
       [111.908303, 28.000189],
       [111.900097, 27.981343],
       [111.912624, 27.973803],
       [111.926848, 27.980851],
       [111.936311, 27.991231],
       [111.951246, 27.987079],
       [111.964156, 27.965171],
       [111.982264, 27.954351],
       [111.99025, 27.940962],
       [111.999714, 27.907837],
       [112.017548, 27.906307],
       [112.030513, 27.914398],
       [112.04408, 27.912156],
       [112.051246, 27.919208],
       [112.060218, 27.920028],
       [112.087734, 27.908384],
       [112.093095, 27.895919],
    

In [ ]:
data_areas['features'][9]['properties']['name']

In [ ]:
# Filenames of temp json files
stations_json_path = os.path.join(temp_dir, 'points.json')
areas_json_path = os.path.join(temp_dir, 'polygons.json')
# Save dictionaries into json files
with open(stations_json_path, 'w') as point_json_file:
    json.dump(data_stations, point_json_file, indent=4)
with open(areas_json_path, 'w') as poly_json_file:
    json.dump(data_areas, poly_json_file, indent=4)

## 方法二 用geopandas的方法替代

In [26]:
import geopandas as gpd
# url = 'https://coralreefwatch.noaa.gov/product/vs/vs_polygons.json' # 珊瑚礁观测网站
path  = ("./data/vs_polygons.json")
temp_dir = tempfile.mkdtemp() # 定义临时目录
data = gpd.read_file(path)

CRSError: Invalid projection: epsg:4326: (Internal Proj Error: proj_create: no database context specified)

In [ ]:
data.geometry.type.unique()

In [ ]:
data[data.geometry.type == 'Point']

In [ ]:
data[data.geometry.type != 'Point']

In [ ]:
stations_json_path = os.path.join(temp_dir, 'points.json')
areas_json_path = os.path.join(temp_dir, 'polygons.json')

In [ ]:
print(stations_json_path)
print(areas_json_path)

In [10]:
# 选出geometry为点和面数据分开保存
data[data.geometry.type == 'Point'].to_file(stations_json_path, driver='GeoJSON')
data[data.geometry.type != 'Point'].to_file(areas_json_path, driver='GeoJSON')

InvalidVersion: Invalid version: '3.4.0e'

## 导入gdb

In [15]:
# Convert JSON files to features
arcpy.conversion.JSONToFeatures(stations_json_path, 'alert_stations')
arcpy.conversion.JSONToFeatures(areas_json_path, 'alert_areas')

<Result 'C:/Temp\\Live.gdb\\alert_areas'>

## 更改符号系统

In [16]:
# Add alert_level field
arcpy.management.AddField('alert_stations', 'alert_level', 'SHORT', field_alias='Alert Level')
arcpy.management.AddField('alert_areas', 'alert_level', 'SHORT', field_alias='Alert Level')

<Result 'alert_areas'>

In [17]:
# Calculate alert_level field
# int(!alert!)是一种动态参数，它将根据不同记录中的字段值来自动计算。而int(alert)则表示将所有记录中的字段值都转换成相同的整型值。
arcpy.management.CalculateField('alert_stations', 'alert_level', "int(!alert!)") # 。 您的表达式将使用 int() 函数将 alert 文本字段的值转换为整型。]

arcpy.management.CalculateField('alert_areas', 'alert_level', "int(!alert!)")

<Result 'alert_areas'>

In [19]:
# Symbology
p = arcpy.mp.ArcGISProject("CURRENT")
m = p.listMaps('地图')[0]
print(m)
# Points
points_lyr = m.listLayers('alert_*')[0]
points_sym = points_lyr.symbology # 通过 Symbology 类，您可以访问要素图层的渲染器或栅格图层的着色器。

In [24]:
points_sym.renderer.type

'GraduatedSymbolsRenderer'

### symbology说明
典型的工作流是参考图层的 symbology，进行更改，然后将更改应用回图层。

重要的是，应首先确保图层支持渲染器或着色器并在对其进行更新之前了解其类型。此时，可使用 hasattr 函数。

目前，arcpy.mp 中支持以下要素图层渲染器：SimpleRenderer、GraduatedColorsRenderer、GraduatedSymbolsRenderer、UnclassedColorsRenderer 和 UniqueValueRenderer。

支持以下栅格着色器：RasterClassifyColorizer、RasterStretchColorizer 和 RasterUniqueValueColorizer。

接下来，您将更新渲染器。 
当前，符号系统使用 SimpleRenderer 类型（单一符号）进行渲染。 要基于字段以不同的方式符号化要素，也需要将渲染器更改为 GraduatedSymbolsRenderer（分级符号）。
您还要将渲染器设置为使用 alert_level 字段作为分类字段，并将数据分为 4 类（0 到 1、1 到 2、2 到 3 和 3 到 4）。 您将针对每一类设置其大小和颜色，使具有较高警报级别的类的大小增加，颜色从蓝色变为红色。

In [20]:
# Always change to the GraduatedSymbolsRenderer from the SimpleRenderer
if points_sym.renderer.type != 'SimpleRenderer':
    points_sym.updateRenderer('SimpleRenderer')
points_sym.updateRenderer('GraduatedSymbolsRenderer') # updateRenderer 方法用于更改图层渲染器。
points_sym.renderer.classificationField = 'alert_level' # renderer获取要素图层的渲染器对象。支持的类型为：GraduatedColorsRenderer、GraduatedSymbolsRenderer、SimpleRenderer 或 UniqueValueRenderer。
points_sym.renderer.breakCount = 4

points_labels = ['0 - 1', '> 1 - 2', '> 2 - 3', '> 3 - 4']
points_upperBounds = [1, 2, 3, 4]
points_sizes = [6, 16.50, 27, 37.50] 
layers_colors = [{'RGB': [5, 113, 176, 40]}, {'RGB': [146, 197, 222, 50]},
                   {'RGB': [244, 165, 130, 50]}, {'RGB': [202, 0, 32, 30]}]

In [21]:
# 您将创建一个 for 循环以将您指定的大小和颜色应用到各个类。 您的类包含在 1 到 4 的数字列表中。 您将使用 i（表示整型）作为占位符变量以代表用数字表示的类。
for i in range(4):
    item = points_sym.renderer.classBreaks[i]
    item.symbol.applySymbolFromGallery('Circle', 1)
    item.label = points_labels[i]
    item.upperBound = points_upperBounds[i]
    item.symbol.size = points_sizes[i]
    item.symbol.color = layers_colors[i]
# Update
points_lyr.symbology = points_sym

TypeError: index out of range

In [26]:
# 使用类似的代码，您可以更改 alert_areas 图层的符号系统。 首先，您将为图层 (polygons_lyr) 及其符号系统 (polygons_sym) 创建变量。

# 运行以下行：
# Polygons
polygons_lyr = m.listLayers('alert_*')[1]
polygons_sym = polygons_lyr.symbology

# Always change to the GraduatedSymbolsRenderer from the SimpleRenderer
if polygons_sym.renderer.type != 'SimpleRenderer':
    polygons_sym.updateRenderer('SimpleRenderer') 
polygons_sym.updateRenderer('GraduatedColorsRenderer')
polygons_sym.renderer.classificationField = 'alert_level'
polygons_sym.renderer.breakCount = 4

polygons_labels = ['0 - 1', '> 1 - 2', '> 2 - 3', '> 3 - 4']
polygons_upperBounds = [1, 2, 3, 4]
layers_colors = [{'RGB': [5, 113, 176, 40]}, {'RGB': [146, 197, 222, 50]},
                   {'RGB': [244, 165, 130, 50]}, {'RGB': [202, 0, 32, 30]}]

for i in range(4):
    item = polygons_sym.renderer.classBreaks[i]
    item.label = polygons_labels[i]
    item.upperBound = polygons_upperBounds[i]
    item.symbol.color = layers_colors[i]

# Update
polygons_lyr.symbology = polygons_sym

In [29]:
# 添加虚线效果
l_cim = polygons_lyr.getDefinition('V2')
l_cim

In [32]:
# Modify a few boolean properties
l_cim.showMapTips = True  #Turn on map tips for bubble tips to appear
l_cim.selectable = False  #Set the layer to not be selectable
l_cim.expanded = True     #Expand the Layer in the Contents pane

In [34]:
# Push the changes back to the layer object
polygons_lyr.setDefinition(l_cim)

# Save changes
p = arcpy.mp.ArcGISProject('CURRENT')
p.save()